## 경기스코어 크롤링

### 한화이글스홈페이지

In [10]:
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.common.exceptions import NoSuchElementException, ElementClickInterceptedException, TimeoutException, UnexpectedAlertPresentException
from selenium.webdriver.common.alert import Alert
import time
import pandas as pd

# 웹드라이버 설정
driver = webdriver.Chrome()

# URL로 접속
url = 'https://www.hanwhaeagles.co.kr/GA/GE/PCFAGE02.do?gmKey=20240311HTHH0&ymd=20240311'
driver.get(url)
driver.maximize_window()

# 데이터 저장 리스트
data = []

# 경고창 처리 및 새로 고침 함수
def handle_alert_and_refresh():
    try:
        # 경고창이 있을 경우 처리
        alert = Alert(driver)
        alert.accept()  # 경고창을 닫기 (OK 버튼 클릭)
        print("경고창이 처리되었습니다.")
        
        # 페이지 새로고침
        driver.refresh()
        print("페이지를 새로고침했습니다.")
        
        # 페이지 새로고침 후 로딩 대기
        WebDriverWait(driver, 20).until(
            EC.presence_of_element_located((By.CSS_SELECTOR, '#container'))  # 로딩 대기
        )
        print("페이지가 새로고침 후 로드되었습니다.")
    except Exception as e:
        print(f"경고창 없음. 오류: {e}")

def extract_game_data():
    # 날짜와 요일, 시간, 장소
    date = driver.find_element(By.CSS_SELECTOR, '#container > section > div.tab_conts > div.select_date.view > strong > span.date').text
    day = driver.find_element(By.CSS_SELECTOR, '#container > section > div.tab_conts > div.select_date.view > strong > span.day').text
    playtime = driver.find_element(By.CSS_SELECTOR, '#container > section > div.tab_conts > div.schedule_view > div.schedule_head > div.now_rs > div.row > span.time').text
    place = driver.find_element(By.CSS_SELECTOR, '#container > section > div.tab_conts > div.schedule_view > div.schedule_head > div.now_rs > div.row > span.place').text

    # 팀1,2이름 추출
    team1_name = driver.find_element(By.CSS_SELECTOR, '#container > section > div.tab_conts > div.schedule_det > div.his_wrap.team > div > table > tbody > tr:nth-child(1) > th').text
    team2_name = driver.find_element(By.CSS_SELECTOR, '#container > section > div.tab_conts > div.schedule_det > div.his_wrap.team > div > table > tbody > tr:nth-child(2) > th').text

    # 팀1,2 점수 추출
    team1_score = driver.find_element(By.CSS_SELECTOR, '#container > section > div.tab_conts > div.schedule_view > div.schedule_head > div.club.left > strong').text
    team2_score = driver.find_element(By.CSS_SELECTOR, '#container > section > div.tab_conts > div.schedule_view > div.schedule_head > div.club.right > strong').text

    # 이닝별 점수 추출
    innings_scores = []
    for inning in range(1, 13):
        try:
            # 각 이닝의 점수를 추출
            team1_inning_score = driver.find_element(By.CSS_SELECTOR, f'#container > section > div.tab_conts > div.schedule_det > div.his_wrap.team > div > table > tbody > tr:nth-child(1) > td:nth-child({inning + 1})').text
            team2_inning_score = driver.find_element(By.CSS_SELECTOR, f'#container > section > div.tab_conts > div.schedule_det > div.his_wrap.team > div > table > tbody > tr:nth-child(2) > td:nth-child({inning + 1})').text
            innings_scores.append((team1_inning_score, team2_inning_score))
        except NoSuchElementException:
            # 이닝에 대한 데이터가 없을 경우 처리
            innings_scores.append((None, None))

    # 추출한 데이터를 리스트에 저장
    data.append([date, day, playtime, place, team1_name, team2_name, team1_score, team2_score] + [item for sublist in innings_scores for item in sublist])

    return date  # 날짜를 반환

# 페이지 새로고침 후 첫 번째 데이터 추출
handle_alert_and_refresh()  # 페이지를 새로고침하고 경고창을 처리합니다.

# 첫 번째 페이지에서 데이터 추출
date = extract_game_data()

# 전체 144경기를 크롤링하기 위해 반복문 사용
game_count = 1  # 첫 번째 경기를 이미 크롤링했으므로 1부터 시작

while True:
    try:
        # "다음 경기" 버튼이 클릭 가능한 상태인지 확인
        next_game_button = WebDriverWait(driver, 10).until(
            EC.element_to_be_clickable((By.CSS_SELECTOR, '#move_next_game'))
        )

        # 버튼 클릭
        next_game_button.click()

        # 경고창이 있는 경우 처리
        handle_alert_and_refresh()

        # 페이지가 로딩될 때까지 기다리기
        WebDriverWait(driver, 20).until(
            EC.presence_of_element_located((By.CSS_SELECTOR, '#container'))  # 특정 요소가 로드되었는지 확인
        )

        # 데이터 추출
        current_date = extract_game_data()  # 새로 추출된 날짜 값을 확인

        # 만약 날짜가 2025-03-22이면 크롤링 종료
        if current_date == "2025-03-22":
            print("지정된 날짜에 도달했습니다. 크롤링을 종료합니다.")
            break

        game_count += 1
        print(f"{game_count}번째 경기 크롤링 완료")

    except ElementClickInterceptedException:
        print("버튼이 다른 요소에 의해 가려져서 클릭할 수 없습니다. 기다리겠습니다.")
        time.sleep(2)
        continue
    except TimeoutException:
        print("페이지 로딩 시간이 초과되었습니다. 기다린 후 재시도합니다.")
        time.sleep(5)
        continue
    except NoSuchElementException:
        print("모든 경기를 크롤링했습니다.")
        break
    except UnexpectedAlertPresentException:
        print("경고창이 발생했습니다. 처리 중...")
        handle_alert_and_refresh()  # 경고창 발생 시 처리
        continue

# 데이터 프레임 생성
columns = ['날짜', '요일', '경기시간', '장소', '팀1명',  '팀2명', '팀1 점수', '팀2 점수']
columns += [f'팀1 {i}이닝점수' for i in range(1, 13)] + [f'팀2 {i}이닝점수' for i in range(1, 13)]
df = pd.DataFrame(data, columns=columns)

# CSV로 저장
df.to_csv('game_scores_homepage.csv', index=False, encoding='utf-8-sig')

print("크롤링 완료! CSV로 저장되었습니다.")

# 브라우저 종료
driver.quit()


경고창 없음. 오류: Message: no such alert
  (Session info: chrome=131.0.6778.205)
Stacktrace:
	GetHandleVerifier [0x00007FF6E8B2FB05+28789]
	(No symbol) [0x00007FF6E8A986E0]
	(No symbol) [0x00007FF6E8935759]
	(No symbol) [0x00007FF6E8925C28]
	(No symbol) [0x00007FF6E894A721]
	(No symbol) [0x00007FF6E895B365]
	(No symbol) [0x00007FF6E89AF4ED]
	(No symbol) [0x00007FF6E895ABE5]
	(No symbol) [0x00007FF6E89AF1E3]
	(No symbol) [0x00007FF6E897A938]
	(No symbol) [0x00007FF6E897BAA1]
	GetHandleVerifier [0x00007FF6E8E6933D+3410093]
	GetHandleVerifier [0x00007FF6E8E7E7DD+3497293]
	GetHandleVerifier [0x00007FF6E8E72A73+3448803]
	GetHandleVerifier [0x00007FF6E8BF7BBB+848171]
	(No symbol) [0x00007FF6E8AA3C3F]
	(No symbol) [0x00007FF6E8A9F6E4]
	(No symbol) [0x00007FF6E8A9F87D]
	(No symbol) [0x00007FF6E8A8ED49]
	BaseThreadInitThunk [0x00007FFAE2CD259D+29]
	RtlUserThreadStart [0x00007FFAE368AF38+40]

경고창 없음. 오류: Message: no such alert
  (Session info: chrome=131.0.6778.205)
Stacktrace:
	GetHandleVerifier [0x0

## 타자기록 크롤링

In [2]:
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.common.exceptions import NoSuchElementException, ElementClickInterceptedException, TimeoutException, UnexpectedAlertPresentException
from selenium.webdriver.common.alert import Alert
import time
import pandas as pd

# 웹드라이버 설정
driver = webdriver.Chrome()

# URL로 접속
url = 'https://www.hanwhaeagles.co.kr/GA/GE/PCFAGE02.do?gmKey=20240311HTHH0&ymd=20240311'
driver.get(url)
driver.maximize_window()

# 데이터 저장 리스트
data2 = []

# 경고창 처리 및 새로 고침 함수
def handle_alert_and_refresh():
    try:
        # 경고창이 있을 경우 처리
        alert = Alert(driver)
        alert.accept()  # 경고창을 닫기 (OK 버튼 클릭)
        print("경고창이 처리되었습니다.")
        
        # 페이지 새로고침
        driver.refresh()
        print("페이지를 새로고침했습니다.")
        
        # 페이지 새로고침 후 로딩 대기
        WebDriverWait(driver, 20).until(
            EC.presence_of_element_located((By.CSS_SELECTOR, '#container'))  # 로딩 대기
        )
        print("페이지가 새로고침 후 로드되었습니다.")
    except Exception as e:
        print(f"경고창 없음. 오류: {e}")
        
def extract_game_data():
    # 날짜와 요일, 시간, 장소소
    date = driver.find_element(By.CSS_SELECTOR, '#container > section > div.tab_conts > div.select_date.view > strong > span.date').text
    h_players_data = driver.find_elements(By.CSS_SELECTOR, '#container > section > div.tab_conts > div.schedule_det > div.his_wrap.hitter > div > table > tbody > tr')

    for player in h_players_data:
        player_name = player.find_element(By.CSS_SELECTOR, 'th').text.strip()  # 선수명
        player_2 = player.find_element(By.CSS_SELECTOR, 'td:nth-child(2)').text.strip()  # 타석
        player_3 = player.find_element(By.CSS_SELECTOR, 'td:nth-child(3)').text.strip()  # 타수
        player_4 = player.find_element(By.CSS_SELECTOR, 'td:nth-child(4)').text.strip()  # 득점
        player_5 = player.find_element(By.CSS_SELECTOR, 'td:nth-child(5)').text.strip()  # 안타
        player_6 = player.find_element(By.CSS_SELECTOR, 'td:nth-child(6)').text.strip()  # 홈런
        player_7 = player.find_element(By.CSS_SELECTOR, 'td:nth-child(7)').text.strip()  # 타점
        player_8 = player.find_element(By.CSS_SELECTOR, 'td:nth-child(8)').text.strip()  # 도루
        player_9 = player.find_element(By.CSS_SELECTOR, 'td:nth-child(9)').text.strip()  # 희타
        player_10 = player.find_element(By.CSS_SELECTOR, 'td:nth-child(10)').text.strip()  # 사사구
        player_11 = player.find_element(By.CSS_SELECTOR, 'td:nth-child(11)').text.strip()  # 삼진진

        # 추출한 데이터를 리스트에 저장
        data2.append([date, player_name, player_2, player_3, player_4,
                     player_5, player_6, player_7, player_8, player_9,
                     player_10, player_11])
    
    # 날짜 값을 반환하여 종료 조건 체크
    return date

# 첫 번째 페이지에서 데이터 추출
current_date = extract_game_data()

# 전체 144경기를 크롤링하기 위해 반복문 사용
game_count = 1  # 첫 번째 경기를 이미 크롤링했으므로 1부터 시작

while True:
    try:
        # "다음 경기" 버튼이 클릭 가능한 상태인지 확인
        next_game_button = WebDriverWait(driver, 10).until(
            EC.element_to_be_clickable((By.CSS_SELECTOR, '#move_next_game'))
        )

        # 버튼 클릭
        next_game_button.click()

        # 경고창이 있는 경우 처리
        handle_alert_and_refresh()

        # 페이지가 로딩될 때까지 기다리기
        WebDriverWait(driver, 20).until(
            EC.presence_of_element_located((By.CSS_SELECTOR, '#container'))  # 특정 요소가 로드되었는지 확인
        )

        # 데이터 추출
        current_date = extract_game_data()  # 새로 추출된 날짜 값을 확인

        # 만약 날짜가 2025-03-22이면 크롤링 종료
        if current_date == "2025-03-22":
            print("지정된 날짜에 도달했습니다. 크롤링을 종료합니다.")
            break

        game_count += 1
        print(f"{game_count}번째 경기 크롤링 완료")

    except ElementClickInterceptedException:
        print("버튼이 다른 요소에 의해 가려져서 클릭할 수 없습니다. 기다리겠습니다.")
        time.sleep(2)
        continue
    except TimeoutException:
        print("페이지 로딩 시간이 초과되었습니다. 기다린 후 재시도합니다.")
        time.sleep(5)
        continue
    except NoSuchElementException:
        print("모든 경기를 크롤링했습니다.")
        break
    except UnexpectedAlertPresentException:
        print("경고창이 발생했습니다. 처리 중...")
        handle_alert_and_refresh()  # 경고창 발생 시 처리
        continue

# DataFrame 생성
columns = ['날짜', '선수명', '타석', '타수', '득점', '안타', '홈런', '타점', '도루', '희타', '사사구', '삼진']
df2 = pd.DataFrame(data2, columns=columns)

# 브라우저 종료
driver.quit()

# 데이터 CSV로 저장
df2.to_csv('hitter_hanhwa.csv', index=False, encoding='utf-8-sig')
print("크롤링 완료! CSV로 저장되었습니다.")


경고창 없음. 오류: Message: no such alert
  (Session info: chrome=131.0.6778.205)
Stacktrace:
	GetHandleVerifier [0x00007FF73E47FB05+28789]
	(No symbol) [0x00007FF73E3E86E0]
	(No symbol) [0x00007FF73E285759]
	(No symbol) [0x00007FF73E275C28]
	(No symbol) [0x00007FF73E29A721]
	(No symbol) [0x00007FF73E2AB365]
	(No symbol) [0x00007FF73E2FF4ED]
	(No symbol) [0x00007FF73E2AABE5]
	(No symbol) [0x00007FF73E2FF1E3]
	(No symbol) [0x00007FF73E2CA938]
	(No symbol) [0x00007FF73E2CBAA1]
	GetHandleVerifier [0x00007FF73E7B933D+3410093]
	GetHandleVerifier [0x00007FF73E7CE7DD+3497293]
	GetHandleVerifier [0x00007FF73E7C2A73+3448803]
	GetHandleVerifier [0x00007FF73E547BBB+848171]
	(No symbol) [0x00007FF73E3F3C3F]
	(No symbol) [0x00007FF73E3EF6E4]
	(No symbol) [0x00007FF73E3EF87D]
	(No symbol) [0x00007FF73E3DED49]
	BaseThreadInitThunk [0x00007FFAE2CD259D+29]
	RtlUserThreadStart [0x00007FFAE368AF38+40]

2번째 경기 크롤링 완료
경고창 없음. 오류: Message: no such alert
  (Session info: chrome=131.0.6778.205)
Stacktrace:
	GetHandl

## 투수기록 크롤링

In [1]:
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.common.exceptions import NoSuchElementException, ElementClickInterceptedException, TimeoutException, UnexpectedAlertPresentException
from selenium.webdriver.common.alert import Alert
import time
import pandas as pd

# 웹드라이버 설정
driver = webdriver.Chrome()

# URL로 접속
url = 'https://www.hanwhaeagles.co.kr/GA/GE/PCFAGE02.do?gmKey=20240311HTHH0&ymd=20240311'
driver.get(url)
driver.maximize_window()

# 데이터 저장 리스트
data2 = []

# 경고창 처리 및 새로 고침 함수
def handle_alert_and_refresh():
    try:
        # 경고창이 있을 경우 처리
        alert = Alert(driver)
        alert.accept()  # 경고창을 닫기 (OK 버튼 클릭)
        print("경고창이 처리되었습니다.")
        
        # 페이지 새로고침
        driver.refresh()
        print("페이지를 새로고침했습니다.")
        
        # 페이지 새로고침 후 로딩 대기
        WebDriverWait(driver, 20).until(
            EC.presence_of_element_located((By.CSS_SELECTOR, '#container'))  # 로딩 대기
        )
        print("페이지가 새로고침 후 로드되었습니다.")
    except Exception as e:
        print(f"경고창 없음. 오류: {e}")
        
def extract_game_data():
    # 날짜와 요일, 시간, 장소소
    date = driver.find_element(By.CSS_SELECTOR, '#container > section > div.tab_conts > div.select_date.view > strong > span.date').text
    p_players_data = driver.find_elements(By.CSS_SELECTOR, '#container > section > div.tab_conts > div.schedule_det > div.his_wrap.pitcher > div > table > tbody > tr')




    for player in p_players_data:
        player_name = player.find_element(By.CSS_SELECTOR, 'th').text.strip()  # 선수명
        player_2 = player.find_element(By.CSS_SELECTOR, 'td:nth-child(3)').text.strip()  # 이닝
        player_3 = player.find_element(By.CSS_SELECTOR, 'td:nth-child(4)').text.strip()  # 타자
        player_4 = player.find_element(By.CSS_SELECTOR, 'td:nth-child(5)').text.strip()  # 타수
        player_5 = player.find_element(By.CSS_SELECTOR, 'td:nth-child(6)').text.strip()  # 투구수
        player_6 = player.find_element(By.CSS_SELECTOR, 'td:nth-child(7)').text.strip()  # 피안타
        player_7 = player.find_element(By.CSS_SELECTOR, 'td:nth-child(8)').text.strip()  # 피홈런
        player_8 = player.find_element(By.CSS_SELECTOR, 'td:nth-child(9)').text.strip()  # 희타
        player_9 = player.find_element(By.CSS_SELECTOR, 'td:nth-child(10)').text.strip()  # 볼넷
        player_10 = player.find_element(By.CSS_SELECTOR, 'td:nth-child(11)').text.strip()  # 삼진
        player_11 = player.find_element(By.CSS_SELECTOR, 'td:nth-child(12)').text.strip()  # 실점
        player_12 = player.find_element(By.CSS_SELECTOR, 'td:nth-child(13)').text.strip()  # 자책점



        # 추출한 데이터를 리스트에 저장
        data2.append([date, player_name, player_2, player_3, player_4,
                     player_5, player_6, player_7, player_8, player_9,
                     player_10, player_11,player_12])
    
    # 날짜 값을 반환하여 종료 조건 체크
    return date

# 첫 번째 페이지에서 데이터 추출
current_date = extract_game_data()

# 전체 144경기를 크롤링하기 위해 반복문 사용
game_count = 1  # 첫 번째 경기를 이미 크롤링했으므로 1부터 시작

while True:
    try:
        # "다음 경기" 버튼이 클릭 가능한 상태인지 확인
        next_game_button = WebDriverWait(driver, 10).until(
            EC.element_to_be_clickable((By.CSS_SELECTOR, '#move_next_game'))
        )

        # 버튼 클릭
        next_game_button.click()

        # 경고창이 있는 경우 처리
        handle_alert_and_refresh()

        # 페이지가 로딩될 때까지 기다리기
        WebDriverWait(driver, 20).until(
            EC.presence_of_element_located((By.CSS_SELECTOR, '#container'))  # 특정 요소가 로드되었는지 확인
        )

        # 데이터 추출
        current_date = extract_game_data()  # 새로 추출된 날짜 값을 확인

        # 만약 날짜가 2025-03-22이면 크롤링 종료
        if current_date == "2025-03-22":
            print("지정된 날짜에 도달했습니다. 크롤링을 종료합니다.")
            break

        game_count += 1
        print(f"{game_count}번째 경기 크롤링 완료")

    except ElementClickInterceptedException:
        print("버튼이 다른 요소에 의해 가려져서 클릭할 수 없습니다. 기다리겠습니다.")
        time.sleep(2)
        continue
    except TimeoutException:
        print("페이지 로딩 시간이 초과되었습니다. 기다린 후 재시도합니다.")
        time.sleep(5)
        continue
    except NoSuchElementException:
        print("모든 경기를 크롤링했습니다.")
        break
    except UnexpectedAlertPresentException:
        print("경고창이 발생했습니다. 처리 중...")
        handle_alert_and_refresh()  # 경고창 발생 시 처리
        continue

# DataFrame 생성
columns = ['날짜', '선수명', '이닝','타자','타수','투구수','피안타','피홈런','희타','볼넷','삼진','실점','자책점']
df2 = pd.DataFrame(data2, columns=columns)

# 브라우저 종료
driver.quit()

# 데이터 CSV로 저장
df2.to_csv('pitcher_hanhwa.csv', index=False, encoding='utf-8-sig')
print("크롤링 완료! CSV로 저장되었습니다.")


경고창 없음. 오류: Message: no such alert
  (Session info: chrome=131.0.6778.205)
Stacktrace:
	GetHandleVerifier [0x00007FF73E47FB05+28789]
	(No symbol) [0x00007FF73E3E86E0]
	(No symbol) [0x00007FF73E285759]
	(No symbol) [0x00007FF73E275C28]
	(No symbol) [0x00007FF73E29A721]
	(No symbol) [0x00007FF73E2AB365]
	(No symbol) [0x00007FF73E2FF4ED]
	(No symbol) [0x00007FF73E2AABE5]
	(No symbol) [0x00007FF73E2FF1E3]
	(No symbol) [0x00007FF73E2CA938]
	(No symbol) [0x00007FF73E2CBAA1]
	GetHandleVerifier [0x00007FF73E7B933D+3410093]
	GetHandleVerifier [0x00007FF73E7CE7DD+3497293]
	GetHandleVerifier [0x00007FF73E7C2A73+3448803]
	GetHandleVerifier [0x00007FF73E547BBB+848171]
	(No symbol) [0x00007FF73E3F3C3F]
	(No symbol) [0x00007FF73E3EF6E4]
	(No symbol) [0x00007FF73E3EF87D]
	(No symbol) [0x00007FF73E3DED49]
	BaseThreadInitThunk [0x00007FFAE2CD259D+29]
	RtlUserThreadStart [0x00007FFAE368AF38+40]

2번째 경기 크롤링 완료
경고창 없음. 오류: Message: no such alert
  (Session info: chrome=131.0.6778.205)
Stacktrace:
	GetHandl